In [ ]:
import matplotlib.pyplot as plt
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from PIL import Image
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
import os
from huggingface_hub import login


In [ ]:
os.environ['HUGGINGFACE_TOKEN'] = 'your hugging face token'
login(os.environ['HUGGINGFACE_TOKEN'])

In [ ]:
def show_stats(df_):
    df = df_.copy()
    
    # Plotting the frequency of categories
    plt.figure(figsize=(10, 6))
    sns.countplot(y=df['meme_anxiety_categories'], order=df['meme_anxiety_categories'].value_counts().index)
    
    plt.title('Frequency of Meme Anxiety Categories')
    plt.xlabel('Count')
    plt.ylabel('Categories')
    plt.show()

In [ ]:

test_df =pd.read_csv('path to your main anxiety dataset test csv')
train_df =pd.read_csv('path to your main anxiety dataset train csv')



In [ ]:
train_df['meme_anxiety_categories'][0]

In [ ]:
train_df['meme_anxiety_categories'].value_counts()

In [ ]:
test_df.head()

In [6]:
train_df.dropna(inplace = True)
test_df.dropna(inplace = True)

In [7]:
train_df.drop(columns='Unnamed: 0',inplace = True)
test_df.drop(columns='Unnamed: 0',inplace = True)

In [ ]:
show_stats(train_df)
show_stats(test_df)

In [13]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
def split_ocr(ocr, ocr_word_limit):
        ocr_words = ocr.split()
        
        if len(ocr_words) > ocr_word_limit:
            ocr = ' '.join(ocr_words[:ocr_word_limit])
        return ocr

In [ ]:
import re

def detect_formatting(triples):
    """ Detects the formatting used for section titles. """
    if '###' in triples:
        return 'hash'
    elif '**' in triples:
        return 'star'
    else:
        return 'unknown'

def process_triples(triples):
    """ Process the triples based on the detected formatting. """
    formatting = detect_formatting(triples)
    
    # Define the sections you want to keep
    sections_to_keep = ['Cause-Effect', 'Figurative Understanding', 'Mental State']
    
    if formatting == 'hash':
        # Normalize and format the text for hash-based sections
        normalized_triples = re.sub(r'\d+\.\s*', '', triples)  # Remove numbering
        sections = re.split(r'###\s*(.*)', normalized_triples)
    elif formatting == 'star':
        # Normalize and format the text for star-based sections
        normalized_triples = re.sub(r'\d+\.\s*', '', triples)  # Remove numbering
        normalized_triples = re.sub(r'\*\*\s*', '**', normalized_triples)  # Standardize '**'
        sections = re.split(r'(\*\*.*?\*\*)', normalized_triples)
    else:
        return "Unknown formatting detected"
    
    # Filter and keep only the desired sections
    reduced_triples = []
    
    i = 1
    while i < len(sections):
        if formatting == 'hash':
            section_title = sections[i].strip()
            if section_title in sections_to_keep and i + 1 < len(sections):
                clean_text = sections[i+1].strip()
                reduced_triples.append(f"{section_title.lower()} : \n   {clean_text}")
        elif formatting == 'star':
            section_title = sections[i].strip("**: ").strip()
            if section_title in sections_to_keep and i + 1 < len(sections):
                clean_text = sections[i+1].strip()
                clean_text = re.sub(r'^\d+\.\s*', '', clean_text, flags=re.MULTILINE).strip()
                reduced_triples.append(f"{section_title.lower()} : \n   {clean_text}")
        i += 2
    
    # Return the formatted sections
    if reduced_triples:
        formated_string = '\n'.join(reduced_triples).strip().replace('-', '').replace(' : \n   :', ' : ')
        return formated_string
    else:
        return "No relevant sections found"

# Apply the function to the 'triples' column
train_df['triples'] = train_df['triples'].apply(process_triples)
test_df['triples'] = test_df['triples'].apply(process_triples)

print('------------------------------------------------')
print(train_df['triples'][433])


In [ ]:
print(train_df['triples'][315])

In [17]:
empty_strings_count = (train_df['triples'] == "").sum()

## Preparing Input

In [ ]:


train_df['labels'] = None
test_df['labels'] = None


le = LabelEncoder()
train_df['labels'] = le.fit_transform(train_df['meme_anxiety_categories'])
test_df['labels'] = le.transform(test_df['meme_anxiety_categories'])


def prompt_it(df, n_shot=1):
    df['prompt'] = None
    ocr_word_limit = 75
    prompts = []
    for n, (ocr, desc, indices) in enumerate(zip(df['ocr_text'], df['triples'], df['indices'])):
        indices = eval(indices)

        print(indices)
        #setting up the data for few shot examples:
        _1_shot , _2_shot, _3_shot = indices[0], indices[1], indices[2]
        _1_shot_ocr, _1_shot_desc, _1_shot_labels = train_df['ocr_text'][_1_shot], train_df['triples'][_1_shot], train_df['meme_anxiety_categories'][_1_shot]
        # _2_shot_ocr, _2_shot_desc, _2_shot_labels = train_df['ocr_text'][_2_shot], train_df['triples'][_2_shot], train_df['meme_anxiety_categories'][_2_shot]
        # _3_shot_ocr, _3_shot_desc, _3_shot_labels = train_df['ocr_text'][_3_shot], train_df['triples'][_3_shot], train_df['meme_anxiety_categories'][_3_shot]
        
        ocr = split_ocr(ocr, ocr_word_limit)
        _1_shot_ocr = split_ocr(_1_shot_ocr, ocr_word_limit)
        # _2_shot_ocr = split_ocr(_2_shot_ocr, ocr_word_limit)
        # _3_shot_ocr = split_ocr(_3_shot_ocr, ocr_word_limit)
        
        prompt = f'''#System: You specialize in analyzing mental health behaviors through social media posts. Your task is to classify the mental health issue depicted in a person's post from the following categories: ['Difficulty Relaxing', 'Excessive Worry', 'Impending Doom', 'Irritatbily', 'Lack of Worry Control', 'Nervousness', 'Restlessness'].

##Example1: 
<|ocr_text|>{_1_shot_ocr}
<|commonsense figurative explaination|>\n{_1_shot_desc}
The mental health disorder of the person for this post is : {_1_shot_labels}

###Your_turn:
<|ocr_text|>{ocr}
<|commonsense figurative explaination|>\n{desc}
The mental health disorder of the person for this post is : '''
        
        prompts.append(prompt)
    
    df['prompt'] = prompts

    return df

train_df = prompt_it(train_df)
test_df = prompt_it(test_df)

class MemeDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.texts = df['prompt']
        self.labels = df['labels']
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'targets': torch.tensor(label, dtype=torch.long)
        }

print(list(le.classes_))
test_df['labels'][0:9]

In [ ]:
print(train_df['prompt'][332])

In [ ]:
print(train_df['labels'][332])

In [ ]:
train_df.loc[train_df['sample_id'] == 'TR-1027']

## Mental Bart

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device_ids = [1] 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
model_name = 'Tianlin668/MentalBART'

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Freeze all layers except the last two layers
for name, param in model.named_parameters():
    if "model.encoder.layers" in name or "model.decoder.layers" in name:
        layer_num = int(name.split('.')[3])
        if layer_num < 10:  # Assuming BART has 12 layers
            param.requires_grad = False

# Wrap the model with DataParallel
model = torch.nn.DataParallel(model, device_ids=device_ids)
model = model.to(f'cuda:{device_ids[0]}')



## Counting the maximum token in the input 

In [ ]:
def max_tokens(df):
  temp = pd.DataFrame(df)
  temp['max_token_count'] = temp['prompt'].apply(lambda x: len(tokenizer.encode(x)))

  # Find the maximum token count across all rows
  max_tokens = temp['max_token_count'].max()

  print("Maximum token count:", max_tokens)
  return temp['max_token_count']


train = max_tokens(train_df)
test = max_tokens(test_df)

# Calculate and print the median
train_median = train.median()
train_mean = train.mean()
test_median = test.median()
test_mean = test.mean()

print("Median token count (train):", train_median)
print("Mean token count (train):", train_mean)

print("Median token count (test):", test_median)
print("Mean token count (test):", test_mean)

## Training Loop

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import warnings
from sklearn.exceptions import UndefinedMetricWarning
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import torch.nn.functional as F

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
# set_seed(42)
train_dataset = MemeDataset(train_df, tokenizer, 1024)
val_dataset = MemeDataset(test_df, tokenizer, 1024)
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Model optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

# Training function
def train_model(training_loader, model, optimizer, criterion):
    model.train()
    train_losses = []
    loop = tqdm(enumerate(training_loader), total=len(training_loader), leave=True, colour='GREEN')
    for batch_idx, data in loop:
        ids = data['input_ids'].to(device)
        mask = data['attention_mask'].to(device)
        targets = data['targets'].to(device)  # Ensure targets are class indices, not one-hot
        
        optimizer.zero_grad()
        outputs = model(ids, attention_mask=mask).logits
        # probabilities = F.softmax(outputs, dim=1)  # Apply softmax to get probabilities
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        loop.set_postfix(loss=loss.item())
    return np.mean(train_losses)


# Evaluation function
def eval_model(validation_loader, model, criterion):
    model.eval()
    val_targets = []
    val_outputs = []
    val_losses = []
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader):
            ids = data['input_ids'].to(device)
            mask = data['attention_mask'].to(device)
            targets = data['targets'].to(device)  # Ensure targets are class indices
            
            outputs = model(ids, attention_mask=mask).logits
            # probabilities = F.softmax(outputs, dim=1)  # Apply softmax to get probabilities
            loss = criterion(outputs, targets)
            val_losses.append(loss.item())
            
            val_targets.extend(targets.cpu().numpy())
            val_outputs.extend(torch.argmax(outputs, dim=1).cpu().numpy())  # Convert logits to class predictions

    val_targets = np.array(val_targets)
    val_outputs = np.array(val_outputs)
    
    accuracy = accuracy_score(val_targets, val_outputs)
    
    # Ensure that target_names and unique_classes match
    target_names = ['Restlessness','Nervousness', 'Impending Doom', 'Difficulty Relaxing', 'Lack of Worry Control', 'Excessive Worry']
    
    report = classification_report(
        val_targets, val_outputs, 
        target_names=target_names, 
        output_dict=True, 
        # labels=7
    )
    
    return report, val_targets, val_outputs, np.mean(val_losses), accuracy


# Main training loop
EPOCHS = 10
history = defaultdict(list)
best_accuracy = 0

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}/{EPOCHS}')
    train_loss = train_model(train_loader, model, optimizer, criterion)
    report, val_targets, val_outputs_binary, val_loss, accuracy = eval_model(val_loader, model, criterion)
    
    val_f1_macro = report['macro avg']['f1-score']
    val_f1_weighted = report['weighted avg']['f1-score']
    print(f"\n||Validation Accuracy: {accuracy:.4f} |Validation F1 Macro: {val_f1_macro:.4f}| |Validation F1 Weighted: {val_f1_weighted:.4f}|")
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_f1_weighted'].append(val_f1_weighted)
    
    if val_f1_macro > best_accuracy:
        best_accuracy = val_f1_macro

        print("Classification Report:\n", classification_report(
            val_targets, val_outputs_binary,
            target_names=['Restlessness','Nervousness', 'Impending Doom', 'Difficulty Relaxing', 'Lack of Worry Control', 'Excessive Worry']
        ))

# Plotting results
fig, axs = plt.subplots(2, 1, figsize=(5, 7))

# Plotting the training and validation losses
axs[0].plot(history['train_loss'], label='Train Loss')
axs[0].plot(history['val_loss'], label='Validation Loss')
axs[0].set_title('Training and Validation Losses')
axs[0].set_ylabel('Loss')
axs[0].set_xlabel('Epoch')
axs[0].legend()
axs[0].grid()

# Plotting the validation F1 scores
axs[1].plot(history['val_f1_macro'], label='Validation F1 Macro')
axs[1].plot(history['val_f1_weighted'], label='Validation F1 weighted')
axs[1].set_title('Validation F1 Scores')
axs[1].set_ylabel('F1 Score')
axs[1].set_xlabel('Epoch')
axs[1].legend()
axs[1].grid()

plt.tight_layout()
plt.show()